In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

drive  sample_data


In [3]:
!ls drive/MyDrive/dataset/

 create_sample_output.py		 leaderboard.json      test.zip
 create_sample_output.py.gdoc		 README.txt	       training.zip
'Information on Problem Statement.pdf'	 sample_result1.json


In [4]:
!cp -r drive/MyDrive/dataset/ .

cp: cannot open 'drive/MyDrive/dataset/create_sample_output.py.gdoc' for reading: Operation not supported


In [5]:
import zipfile
with zipfile.ZipFile("dataset/training.zip","r") as zip_ref:
    zip_ref.extractall("training")

In [6]:
import zipfile
with zipfile.ZipFile("dataset/test.zip","r") as zip_ref:
    zip_ref.extractall(".")

In [7]:
import os
import random 
from shutil import copyfile


In [8]:
train_percentage = 75
test_percentage = 25

In [10]:
os.mkdir('train')
os.mkdir('test')
os.mkdir('train/Positive')
os.mkdir('train/Negative')
os.mkdir('test/Positive')
os.mkdir('test/Negative')

In [12]:
pos_class_files = os.listdir('training/Positive')
negative_class_files = os.listdir('training/Negative')
pc = len(pos_class_files)
nc = len(negative_class_files)
pc_train = int((train_percentage/100)*pc)
pc_test = pc - pc_train
nc_train = int((train_percentage/100)*nc)
nc_test = nc - nc_train
random.shuffle(pos_class_files)
random.shuffle(negative_class_files)

for i in range(pc_train):
    copyfile(f'./training/Positive/{pos_class_files[i]}',f'./train/Positive/{pos_class_files[i]}')

for i in range(pc_train,pc):
    copyfile(f'./training/Positive/{pos_class_files[i]}',f'./test/Positive/{pos_class_files[i]}')

for i in range(nc_train):
    copyfile(f'./training/Negative/{negative_class_files[i]}',f'./train/Negative/{negative_class_files[i]}')

for i in range(nc_train,nc):
    copyfile(f'./training/Negative/{negative_class_files[i]}',f'./test/Negative/{negative_class_files[i]}')



In [3]:

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import cv2
import numpy as np
import os
from PIL import Image
import numpy as np

In [7]:
from google.colab.patches import cv2_imshow
x = []
files = os.listdir("training/Negative")
for i in files:
    img = cv2.imread(f"training/Negative/{i}")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x.append(gray)



files = os.listdir("training/Positive")
for i in files:
    img = cv2.imread(f"training/Positive/{i}")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x.append(gray)

In [17]:
x = np.array(x)
y = []
for i in range(0,4450):
  y.append(int(0)) #for bg
for i in range(4450,5875):
  y.append(int(1)) #for text
y = np.array(y)
y.shape



(5875,)

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=314)

In [19]:
x_train = x_train.reshape(-1,64,64,1)
x_test = x_test.reshape(-1,64,64,1)

In [23]:
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import Adam

In [24]:
cnn = models.Sequential([
    layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(64, 64,1)), #layer 1
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'), #layer 2
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'), #layer 3
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'), #layer 4
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    # layers.Dense(80, activation='relu'), ##reducing nodes
    layers.Dense(80, activation='relu'), ##reducing nodes
    layers.Dense(30, activation='relu'), ##reducing nodes
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-2,
#     decay_steps=10000,
#     decay_rate=0.9)
opt = Adam(learning_rate=0.001)
cnn.compile(optimizer=opt, 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [26]:
cnn.fit(x_train, y_train, epochs=10, batch_size = 64, validation_data=(x_test, y_test))


Epoch 1/10
65/65 [==============================] - 15s 227ms/step - loss: 0.5509 - accuracy: 0.7605 - val_loss: 0.6137 - val_accuracy: 0.7476
Epoch 2/10
65/65 [==============================] - 15s 226ms/step - loss: 0.5356 - accuracy: 0.7614 - val_loss: 0.6084 - val_accuracy: 0.7499
Epoch 3/10
65/65 [==============================] - 15s 226ms/step - loss: 0.4990 - accuracy: 0.7629 - val_loss: 0.5649 - val_accuracy: 0.8128
Epoch 4/10
65/65 [==============================] - 15s 227ms/step - loss: 0.4352 - accuracy: 0.8081 - val_loss: 0.5251 - val_accuracy: 0.8741
Epoch 5/10
65/65 [==============================] - 15s 228ms/step - loss: 0.3891 - accuracy: 0.8388 - val_loss: 0.4931 - val_accuracy: 0.8729
Epoch 6/10
65/65 [==============================] - 15s 228ms/step - loss: 0.3445 - accuracy: 0.8614 - val_loss: 0.4290 - val_accuracy: 0.8837
Epoch 7/10
65/65 [==============================] - 15s 226ms/step - loss: 0.2999 - accuracy: 0.8908 - val_loss: 0.3957 - val_accuracy: 0.9002

In [31]:

test_data = {}
files = os.listdir("question")
for i in files:
    img = cv2.imread(f'question/{i}')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    test_data[i]=gray


test_data1 = np.array(list(test_data.values()))
test_data1 = test_data1.reshape(-1,64,64,1)
prediction = cnn.predict(test_data1)


In [34]:

for img,i in zip(list(test_data.keys()),prediction):
  if i <0.5:
    test_data[img] = 0
  else:
    test_data[img] = 1

for elem in test_data : 
  print(elem,test_data[elem])

95.jpg 0
26.jpg 1
40.jpg 0
52.jpg 1
13.jpg 1
23.jpg 1
24.jpg 1
37.jpg 0
14.jpg 0
96.jpg 0
42.jpg 1
82.jpg 1
41.jpg 0
61.jpg 1
63.jpg 1
54.jpg 1
51.jpg 1
53.jpg 1
46.jpg 1
47.jpg 0
59.jpg 0
77.jpg 0
83.jpg 1
27.jpg 0
20.jpg 1
92.jpg 1
9.jpg 0
16.jpg 0
75.jpg 0
72.jpg 0
85.jpg 0
76.jpg 0
94.jpg 1
93.jpg 0
31.jpg 0
68.jpg 0
69.jpg 1
79.jpg 0
91.jpg 0
32.jpg 0
73.jpg 1
97.jpg 0
2.jpg 0
10.jpg 0
48.jpg 1
21.jpg 1
33.jpg 0
17.jpg 0
29.jpg 1
57.jpg 0
90.jpg 1
12.jpg 0
28.jpg 1
30.jpg 1
3.jpg 1
34.jpg 0
8.jpg 0
67.jpg 1
70.jpg 0
86.jpg 0
43.jpg 0
35.jpg 0
6.jpg 0
38.jpg 0
56.jpg 0
88.jpg 0
19.jpg 1
22.jpg 1
84.jpg 1
4.jpg 0
64.jpg 0
18.jpg 0
39.jpg 0
25.jpg 1
98.jpg 0
78.jpg 0
1.jpg 1
44.jpg 0
65.jpg 0
7.jpg 1
71.jpg 0
55.jpg 0
45.jpg 0
66.jpg 0
60.jpg 0
49.jpg 0
5.jpg 1
81.jpg 1
80.jpg 0
58.jpg 0
74.jpg 0
11.jpg 1
62.jpg 0
36.jpg 0
87.jpg 0
15.jpg 1
89.jpg 0
50.jpg 1


In [46]:
import json

with open('output.json', 'w') as outfile:
  json.dump(test_data, outfile)
  
